# TME1 : premier pas en traitement d'image avec Python et Jupyter

> Consignes: le fichier **TME1_Sujet.ipynb** est à déposer sur le site Moodle de l'UE https://moodle-sciences.upmc.fr/moodle-2019/course/view.php?id=4248. Si vous êtes en binôme, renommez-le en **TME1_nom1_nom2.ipynb**.

Pour ce premier TME, qui consiste à réaliser des opérations de base de traitement d'image, nous utiliserons uniquement le language Python natif, sans module particulier et cela pour bien comprendre ce qu'est une image sur le disque et sa représentation en mémoire, une liste de valeurs, organisée en lignes de pixels.

Les deux fonctions suivantes permettent respectivement de lire et d'écrire sur le disque des images au format Portable Grey Map (PGM). Elle utilise le type formel <tt>Image</tt> qui est <tt>tuple[list[int],int,int]</tt>, selon les conventions du cours 1I001. Le premier élément est la liste des pixels de l'image, ordonnée ligne par ligne, le second élément est la longueur d'une ligne (soit le nombre de pixels dans une ligne), et le troisième le nombre de lignes.


In [33]:
def readPGM(file):
    """  str -> tuple[list[int],int,int] | NoneType
    Lit un fichier PGM et retourne la liste des valeurs de 
    l'image et ses dimensions (nombre de colonnes, nombre de lignes).
    """
    fp = open(file,'rb')
    # en mode binaire, readline() retourne un type 'bytes'
    if fp.readline() == b'P5\n':
        while True:
            # lecture d'une ligne, conversion vers str
            line = fp.readline().decode()
            if line[0] != '#': break
        # découpage en mots, puis conversion        
        w,h=line.split()
        w,h=int(w),int(h)
        # Nb de niveaux de gris (pas utile mais il faut le lire)
        l=fp.readline()
        # données
        data = list(fp.read(w*h))
        if len(data) != w*h:
            print ('readPGM: error with ' + file + ': has wrong size')
        fp.close()
        return (data,w,h)
    else:
        print( 'readPGM: error with '+ file + ': unsupported format')
    fp.close()
    return None

def writePGM(image,file):
   """ tuple(list[int],int,int)*str -> NoneType
   Ecrit une image au format PGM 
   """
   data,w,h = image
   fp = open(file,'wb')
   fp.writelines([bytes('P5\n'+str(w)+' '+str(h)+'\n255\n','utf8')])
   fp.write(bytearray(data))
   fp.close()

Pour voir une image, nous utiliserons une commande de votre système (normalement Linux Debian) qui affichera l'image lu depuis le disque. Si vous utilisez un autre système d'exploitation (OSX ou Windows), il faudra utiliser un programme spécifique.     

In [34]:
from os import system

def viewImage(image):
    """ tuple[list[int],int,int] -> NoneType
    Lit et affiche une image depuis le disque
    """
    writePGM(image,'viewimage.pgm')
    system( 'xdg-open viewimage.pgm')
    #system( 'rm -f viewimage.pgm')
    


# Exercice 1: visualisation et histogramme

## 1.1 Voir une image
Écrire un code python qui lit l'image <tt>img/carrefou.pgm</tt> l'affiche à l'écran et imprime ses dimensions.

In [35]:
T,N,M=readPGM("img/oursin.pgm");
viewImage(readPGM("img/oursin.pgm"))
print(N)
print(M)

256
256


## 1.2 Calcul d'histogramme

Écrire une fonction <tt>histogram()</tt> qui prend la liste des valeurs d'une image, un entier <tt>n</tt> et retourne une liste des <tt>n</tt> valeurs représentant l'histogramme de l'image.

In [36]:
def histogram(data,n):
    """ list[int]*int > list[int] """ 
    H=[0]*n
    for p in data:
        H[p]=H[p]+1
    return H

## 1.3 Affichage d'histogramme
Écrire un code Python qui charge une image, calcule son histogramme, et l'affiche l'histogramme sous forme textuelle, par exemple:
<pre>
0 -> 3
1 -> 5
2 -> 10
 ...
</pre>
Dans cet exemple, il faut lire qu'il y a 3 pixels de valeur 0, 5 pixels de valeur 1 et 10 pixels de valeur 2, _etc_.


In [37]:
T,N,M=readPGM("img/oursin.pgm");
H=histogram(T,N*M)
for i in range(256):
    print(i," -> ",H[i])


0  ->  0
1  ->  0
2  ->  0
3  ->  0
4  ->  0
5  ->  0
6  ->  0
7  ->  0
8  ->  0
9  ->  0
10  ->  0
11  ->  0
12  ->  0
13  ->  0
14  ->  0
15  ->  0
16  ->  0
17  ->  0
18  ->  0
19  ->  0
20  ->  0
21  ->  0
22  ->  0
23  ->  0
24  ->  0
25  ->  0
26  ->  0
27  ->  0
28  ->  0
29  ->  0
30  ->  0
31  ->  0
32  ->  0
33  ->  3
34  ->  10
35  ->  16
36  ->  21
37  ->  21
38  ->  9
39  ->  6
40  ->  27
41  ->  55
42  ->  62
43  ->  70
44  ->  76
45  ->  122
46  ->  207
47  ->  481
48  ->  853
49  ->  1468
50  ->  1489
51  ->  2085
52  ->  1939
53  ->  1567
54  ->  1497
55  ->  1637
56  ->  2200
57  ->  2117
58  ->  1838
59  ->  1715
60  ->  1381
61  ->  1186
62  ->  1402
63  ->  1059
64  ->  1207
65  ->  1299
66  ->  1563
67  ->  1319
68  ->  1344
69  ->  1157
70  ->  1068
71  ->  1202
72  ->  1152
73  ->  1563
74  ->  1625
75  ->  1434
76  ->  1300
77  ->  1235
78  ->  1010
79  ->  652
80  ->  783
81  ->  613
82  ->  562
83  ->  550
84  ->  402
85  ->  442
86  ->  466
87  ->  449
88  -

# Exercice 2: étirement d'histogramme
## 2.1 Étirement 
Écrire la fonction <tt>etire()</tt> qui prend un histogramme et retourne un tableau (une liste Python) des correspondance entre niveaux de gris avant et après étirement. L'histogramme doit être étiré sur la dynamique la plus grande, c'est-à-dire [0,255]. Si <tt>table</tt> est la liste retournée et si <tt>table[0]</tt> vaut 2, cela signifie que les pixels ayant le niveau de gris 0 auront alors la valeur 2 après étirement de l'histogramme. Indication: utiliser une des fonctions d'arrondis (<tt>floor</tt>, <tt>ceil</tt>, ...) du module Python <tt>math</tt>

In [46]:
from math import ceil
def etire(his):
    """ list[int] -> list[int] """
    C=[0]*len(his)
    tmin=0
    tmax=0
    i=0
    while(his[i]==0):
        i=i+1
    tmin=i
    
    i=len(his)-1
    while(his[i]==0):
        i=i-1
    tmax=i
    print(tmin,tmax)
    
    for t in range(tmin,tmax+1):
        u=ceil(((t-tmin)/(tmax-tmin))*255)
        C[t]=u
        
    return C

        

## 2.2 Changement des valeurs de l'image
Écrire la fonction <tt>applique()</tt> qui applique la table obtenue par la fonction <tt>etire()</tt> à la liste des valeurs de l'image pour former une nouvelle image son histogramme étiré.

In [47]:
def applique(data,table):
    """ list[int]*list[int] -> list[int] """
    Image=[0]*len(data)
    for p in range(len(data)):
        Image[p]=table[data[p]]
    return Image

## 2.3 Application
Mettre en oeuvre les fonctions précédentes sur les images mises à votre disposition. Visualisez le résultat à l'aide de la fonction <tt>viewImage()</tt>.
Attention: certaines images ont déjà un histogramme étiré ! Comment le vérifier ? (répondre en commentaire dans le code Python ci-dessous).

In [48]:
T,N,M=readPGM("img/oursin.pgm");
H=histogram(T,256)
viewImage((applique(T,etire(H)),N,M))

#Il suffit de voir l'histogramme de chaque image avant l'étirement.
#Si le niveau de gris 0 et 255 sont tout les deux non nulles alors l'image est déjà étirée.

33 174


# Exercice 3: seuillage d'images
## 3.1 Fonction de seuillage
Écrire la fonction <tt>seuillage()</tt> qui prend une liste de pixels <tt>data</tt>, un paramètre de seuil <tt>t</tt>. La liste de valeurs retournées sera consitituée d'une liste de valeur à 0 (pour les valeurs en dessous du seuil <tt>t</tt>) ou 255 (pour les autres valeurs).


In [41]:
def seuillage(data,t):
    """ list[int]*int -> list[int] """
    res=[3000]*len(data)
    for p in range(len(data)):
        if data[p]>t:
            res[p]=255
        else:
            res[p]=0
    return res

## 3.2 Application
Appliquer la fonction de seuillage à différentes images et faire varier le seuil. Utiliser la fonction <tt>viewImage()</tt> pour visualiser ces opérations de seuillage.

In [42]:
T,N,M=readPGM("img/oursin.pgm");
res=seuillage(T,256/4)
viewImage((res,N,M))

# Exercice 4: égalisation d'histogramme 
## 4.1 Égalisation
Écrire la fonction <tt>egalisation()</tt> qui prend un histogramme, applique une égalisation d'histogramme et retourne la table des correspondances entre niveaux de gris avant et après égalisation. L'égalisation d'histogramme consiste à transformer chaque valeur de niveau de gris $k$ en $k'$ selon la formule: $k' = Int\left(\frac{L-1}{N \times M}H_c(k)\right)$ où $H_c$ est l'histogramme cumulé, $N\times M$ la taille de l'image et $L$ la dynamique de l'image.

In [43]:
def egalisation(his):
    """ list[int] -> list[int] """
    Hc=[0]*len(his)
    Hc[0]=his[0]
    for p in range(1,len(his)):
        Hc[p]=Hc[p-1]+his[p]
    
    
    non_null = [i for i in range(len(his)) if his[i] != 0]
    tmin = non_null[0]
    tmax = non_null[-1]
    
    C=[0]*len(his)
    for k in range(len(his)):
        C[k]=int(Hc[k]*(tmax-tmin)/(Hc[-1]))
    return C

## 4.2 Application
Écrire un code Python qui lit une image, réalise son égalisation d'histogramme et affiche la nouvelle image égalisée.

In [45]:
T,N,M=readPGM("img/oursin.pgm")
res=applique(T,egalisation(histogram(T,N)))
viewImage((res,N,M))